## Analisis publicitario

### Carga de datos

In [ ]:
#Importo librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline
plt.style.use('default')
sns.set()

In [ ]:
#Funciones auxiliares
def mostrar_porcentaje_barplot(ax):
    suma = 0
    for p in ax.patches:
        suma += p.get_height()
    for p in ax.patches:
        ax.annotate(str(np.round(100 *(p.get_height() / suma),decimals=2)) + "%", (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')
def mostrar_valores_barplot(ax):
    for p in ax.patches:
        ax.annotate(np.round(p.get_height(),decimals=2), (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
#Cargo el DataFrame
df = pd.read_csv('events.csv', low_memory = False, parse_dates = ['timestamp'], infer_datetime_format = True,
                    dtype = {'event': 'category','condition': 'category','storage': 'category', 'color': 'category', 'staticpage': 'category', 'campaign_source': 'category', 'search_engine': 'category', 'channel': 'category', 'new_vs_returning': 'category', 'region': 'category', 'country': 'category', 'device_type': 'category'})

### Evolución de las campañas a lo largo del tiempo

In [ ]:
n = 5

hits_campanias = df.loc[(df['event']=='ad campaign hit'), ['timestamp', 'person', 'url', 'campaign_source']].copy()
hits_campanias['semana'] = hits_campanias['timestamp'].dt.week - 1
hits_campanias['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(hits_campanias['semana'].mul(7).astype('str') + ' days')
hits_campanias.dropna(subset = ['campaign_source'], inplace = True)

n_mayores_sources = hits_campanias['campaign_source'].value_counts().iloc[0:n]

hits_campanias = pd.crosstab(hits_campanias['semana'], hits_campanias['campaign_source'])
hits_campanias = hits_campanias.loc[:, n_mayores_sources.index]
hits_campanias = hits_campanias.div(hits_campanias.sum(axis=1), axis=0).multiply(100)

ax = hits_campanias.plot(kind = 'area')
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set(xlabel='Semana', ylabel='Porcentaje', title = 'Proporcion de source de '+str(n)+' mayores campañas publicitarias')

### Ventas por campaña

In [ ]:
ad_campaigns = df.loc[(df['event']=='ad campaign hit') | (df['event']=='conversion')]#.drop_duplicates()#[['person','campaign_source']].drop_duplicates().set_index('campaign_source')
ad_campaigns.head()

In [ ]:
ad_campaigns_cool = df.loc[(df['event']=='ad campaign hit'),['person','campaign_source']].drop_duplicates()
ad_campaigns_cool.head()

In [ ]:
personas_que_compraron = df.loc[df['event']=='conversion',['person']].drop_duplicates()
personas_que_compraron['compro'] = True
personas_que_compraron.head()

In [ ]:
compras_por_campania = pd.merge(ad_campaigns_cool, personas_que_compraron, on='person', how='left')
compras_por_campania.head(10)

In [ ]:
compras2 = compras_por_campania.groupby('campaign_source').agg({'person': 'count','compro':'count'})
compras2.head()

In [ ]:
g = compras2['person'].sort_values(ascending=False).head().plot('bar')
g.set_title('Hits por campaña')
g.set_xlabel('Campaña publicitaria')
g.set_ylabel('Cantidad de hits')

In [ ]:
g2 = compras2['compro'].sort_values(ascending=False).plot('bar')
g2.set_title('Cantidad de compras realizadas por campaña')

In [ ]:
#QUITAMOS LOS QUE TIENEN MENOS DE 10 HITS PARA NO DEFORMAR
compras2 = compras2[compras2["person"] > 10]
compras2['porcentaje'] = 100* (compras2['compro']/compras2['person'])
g3 = compras2['porcentaje'].sort_values(ascending=False).plot('bar')
g3.set_title('Porcentaje de compras logradas por campaña publicitaria')

Hay una "campaña publicitaria" llamada onsite que debería ser eliminada del data frame